# JL124 E6C 6-circle example

Compare with data acquired using SPEC

In _hkl_ *E6C* geometry (https://people.debian.org/~picca/hkl/hkl.html#orge5e0490):

<img src="4S+2D.png" alt="E6C geometry" width="30%"/>

* xrays incident on the $\vec{x}$ direction (1, 0, 0)

axis  | moves    | rotation about axis
---   | ---      | ---
mu    | sample   | $\vec{z}$ `[0 0 1]`
omega | sample   | $-\vec{y}$ `[0 -1 0]`
chi   | sample   | $\vec{x}$ `[1 0 0]`
phi   | sample   | $-\vec{y}$ `[0 -1 0]`
gamma | detector | $\vec{z}$ `[0 0 1]`
delta | detector | $-\vec{y}$ `[0 -1 0]`

Huber 6-circle diffractometer at the Advanced Photon Source

<img 
  src="6-circle-diffractometer.jpg" 
  alt="APS 6-circle diffractometer" 
  width="40%"/>


In SPEC *sixc* geometry (https://certif.com/spec_help/sixc.html):

name    | mnemonic   | description
-----   | -----      | -----
Delta   | del        | Detector arm rotation
Theta   | th         | Rotates sample circles
Chi     | chi        | Sample tilt
Phi     | phi        | Sample rotation
Mu      | mu         | Diffractometer rotation
Gamma   | gam        | Out-of-plane detector rotation

> When the azimuthal angle is set to 90 degrees in the azimuth-fixed modes (3, 6, 9 and 12), the incident angle alpha will be equal to the exit angle beta.

In [1]:
# mapping of axis names between E6C and SPEC
AXIS_NAME_MAP = dict(
    # remap so we can use the sixc names
    # E6C  sixc
    mu='mu',  # Diffractometer rotation around vertical axis
    omega='theta',  # Rotates chi around horizontal axis
    chi='chi',  # Rotates phi around beam axis
    phi='phi',  # Sample rotation around horizontal axis (when phi is co-linear with omega)
    gamma='gamma',  # Diffractometer rotation around vertical axis
    delta='delta',  # Detector arm rotation
)

In [2]:
import pyRestTable
from spec2nexus.spec import SpecDataFile

specfile = SpecDataFile("hkl_data/JL124_1_s52.spc")
specscan = specfile.getScan(52)

spec_d = specscan.diffractometer
spec_d.UB = spec_d.geometry_parameters["ub_matrix"][2]

terms = {
    "SPEC file": specfile.specFile,
    "scan #": specscan.scanNum,
    "SPEC scanCmd": specscan.scanCmd,
    "geometry": spec_d.geometry_name,
    "mode": spec_d.mode,
    "lattice": spec_d.lattice,
    "wavelength": spec_d.wavelength,
    "reflection 1": spec_d.reflections[0],
    "reflection 2": spec_d.reflections[1],
    "[UB]": spec_d.UB,
}
tbl = pyRestTable.Table()
tbl.labels = "term value".split()
for k, v in terms.items():
    tbl.addRow((k, v))
print(tbl)

============ ==============================================================================================================================================================================================
term         value                                                                                                                                                                                         
============ ==============================================================================================================================================================================================
SPEC file    /home/33id/data/junelee/20130213/LNO1/JL124_1.spc                                                                                                                                             
scan #       52                                                                                                                                                                         

In [3]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import spec2nexus

import gi
gi.require_version('Hkl', '5.0')
from hkl.diffract import E6C
from hkl.util import Lattice

from ophyd import (PseudoSingle, SoftPositioner)
from ophyd import Component as Cpt

In [4]:
class This(E6C):
    h = Cpt(PseudoSingle, '')
    k = Cpt(PseudoSingle, '')
    l = Cpt(PseudoSingle, '')
    mu = Cpt(SoftPositioner)
    omega = Cpt(SoftPositioner)
    chi = Cpt(SoftPositioner)
    phi = Cpt(SoftPositioner)
    gamma = Cpt(SoftPositioner)
    delta = Cpt(SoftPositioner)

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

        for p in self.real_positioners:
            p._set_position(0)  # give each a starting position

e6c = This("", name="e6c")
print("\n".join(map(str, e6c.real_positioners)))
import yaml
print(yaml.dump(specscan.G))

SoftPositioner(name='e6c_mu', parent='e6c', settle_time=0.0, timeout=None, egu='', limits=(0, 0), source='computed')
SoftPositioner(name='e6c_omega', parent='e6c', settle_time=0.0, timeout=None, egu='', limits=(0, 0), source='computed')
SoftPositioner(name='e6c_chi', parent='e6c', settle_time=0.0, timeout=None, egu='', limits=(0, 0), source='computed')
SoftPositioner(name='e6c_phi', parent='e6c', settle_time=0.0, timeout=None, egu='', limits=(0, 0), source='computed')
SoftPositioner(name='e6c_gamma', parent='e6c', settle_time=0.0, timeout=None, egu='', limits=(0, 0), source='computed')
SoftPositioner(name='e6c_delta', parent='e6c', settle_time=0.0, timeout=None, egu='', limits=(0, 0), source='computed')
G0: 12 0 1 -0.002814275157 0.007740448308 0.9999660821 0 0 0 0 0 0 600 0 1 1 143.6
G1: 3.905 3.905 3.905 90 90 90 1.609010322 1.609010322 1.609010322 90 90 90 0 0 2
  3 0 3 0.003 90 0.5799999712 239.9999477 12.102 12.9945 47.18 90 0.5799999712 239.9999477
  21.77425 15.7375 0.8265616267

In [5]:
# get the UB matrix from the SPEC data
e6c.UB.put(spec_d.UB[[1,2,0], :])
# print(e6c.engine.modes)
e6c.engine.mode = "lifting_detector_omega"
# e6c.calc["chi"].limits = (88, 92)
# e6c.calc["chi"].fit = False
# e6c.calc["chi"].value = 240
e6c.chi.move(240)

# SPEC: del   th mu           chi         phi      gam
# (002) 0.003 90 0.5799999712 239.9999477 12.102   12.9945
# (303) 47.18 90 0.5799999712 239.9999477 21.77425 15.7375
# print(e6c.forward(0.5, 0.5, 1.2))
print(e6c.calc.forward((0, 0, 2)))
modes = []
r_hkl = (3, 0, 3)
for mode in e6c.engine.modes:
    e6c.engine.mode = mode
    try:
        sol = e6c.forward(r_hkl)
        modes.append(mode)
        print(mode, r_hkl, sol)
    except Exception:
        print(mode, r_hkl, "no solution")
r_hkl = (0, 0, 2)
for mode in modes:
    e6c.engine.mode = mode
    try:
        sol = e6c.forward(r_hkl)
        print(mode, r_hkl, sol)
    except Exception:
        print(mode, r_hkl, "no solution")
print(modes)

(PosCalcE6C(mu=0.0, omega=-27.97391316589647, chi=-120.00000000000001, phi=0.0, gamma=-30.996525444813518, delta=-38.344980261923986), PosCalcE6C(mu=0.0, omega=-27.97391316589647, chi=-120.00000000000001, phi=0.0, gamma=149.00347455518647, delta=-141.65501973807605))
bissector_vertical (3, 0, 3) PosCalcE6C(mu=0.0, omega=67.68234833655978, chi=139.5004906975483, phi=129.21870663340866, gamma=0.0, delta=135.36469667311957)
constant_omega_vertical (3, 0, 3) no solution
constant_chi_vertical (3, 0, 3) PosCalcE6C(mu=0.0, omega=-26.264973297127604, chi=-120.00000000000001, phi=9.674485199071988, gamma=0.0, delta=-135.3646966731196)
constant_phi_vertical (3, 0, 3) PosCalcE6C(mu=0.0, omega=31.58777850004291, chi=53.48670337763097, phi=0.0, gamma=0.0, delta=135.36469667311957)
lifting_detector_phi (3, 0, 3) no solution
lifting_detector_omega (3, 0, 3) PosCalcE6C(mu=0.0, omega=-32.00865186003745, chi=-120.00000000000001, phi=0.0, gamma=-13.404723805458783, delta=-137.01416647059162)
lifting_dete

In [6]:
# TODO: compare in a table, as in e4cv

In [7]:
class Diffractometer(E6C):
    h = Cpt(PseudoSingle, '')
    k = Cpt(PseudoSingle, '')
    l = Cpt(PseudoSingle, '')

    # use the SPEC axis names here
    mu = Cpt(SoftPositioner)
    theta = Cpt(SoftPositioner)
    chi = Cpt(SoftPositioner)
    phi = Cpt(SoftPositioner)
    gamma = Cpt(SoftPositioner)
    delta = Cpt(SoftPositioner)

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

        for p in self.real_positioners:
            p._set_position(0)  # give each a starting position

In [8]:
sixc = Diffractometer("", name="sixc")

sixc.calc.physical_axis_names = AXIS_NAME_MAP

```
sample: JL124_1
crystal:  3.905 3.905 3.905 90 90 90
geometry: sixc
mode: 12 (Z-Axis with Azimuth fixed and Chi, Phi set to -Sigma, -Tau)
lambda: 0.8265616267
r1: (0, 0, 2) 0.003 90 0.5799999712 239.9999477 12.102 12.9945
r2: (3, 0, 3) 47.18 90 0.5799999712 239.9999477 21.77425 15.7375
Q: (2.99804, 0.00216068, 2.99661) 47.14125 90.089 0.58 239.94275 21.73025 15.7375
UB: 1.207702707 1.248454819 0.002095582696 
    -1.485612421 0.9118074731 0.003241829804 
    -0.0173752388 0.02282507942 1.651530555
```

In [9]:
lattice = Lattice(
    a=3.905, b=3.905, c=3.905,
    alpha=90.0, beta=90.0, gamma=90.0)

# add the sample to the calculation engine
sixc.calc.new_sample(
    "JL124_1",
    lattice=Lattice(
        a=3.905, b=3.905, c=3.905,
        alpha=90.0, beta=90.0, gamma=90.0)
    )

HklSample(name='JL124_1', lattice=LatticeTuple(a=3.905, b=3.905, c=3.905, alpha=90.0, beta=90.0, gamma=90.0), ux=Parameter(name='None (internally: ux)', limits=(min=-180.0, max=180.0), value=0.0, fit=True, inverted=False, units='Degree'), uy=Parameter(name='None (internally: uy)', limits=(min=-180.0, max=180.0), value=0.0, fit=True, inverted=False, units='Degree'), uz=Parameter(name='None (internally: uz)', limits=(min=-180.0, max=180.0), value=0.0, fit=True, inverted=False, units='Degree'), U=array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]]), UB=array([[ 1.60901032e+00, -9.85234670e-17, -9.85234670e-17],
       [ 0.00000000e+00,  1.60901032e+00, -9.85234670e-17],
       [ 0.00000000e+00,  0.00000000e+00,  1.60901032e+00]]), reflections=[])

In [10]:
sixc.calc.wavelength = 0.8265616267 # angstrom

# SPEC motors (in order): del th chi phi mu gam
# r1: (0, 0, 2) 0.003 90 0.5799999712 239.9999477 12.102 12.9945
# r2: (3, 0, 3) 47.18 90 0.5799999712 239.9999477 21.77425 15.7375

r1 = sixc.calc.sample.add_reflection(
    0, 0, 2, 
    position=sixc.calc.Position(
        delta=0.003, 
        theta=90, 
        chi=0.5799999712,
        phi=239.9999477, 
        mu=12.102, 
        gamma=12.9945,
        )
    )
r2 = sixc.calc.sample.add_reflection(
    3, 0, 3, 
    position=sixc.calc.Position(
        delta=47.18, 
        theta=90, 
        chi=0.5799999712,
        phi=239.9999477, 
        mu=21.77425, 
        gamma=15.7375,
        )
    )
sixc.calc.sample.compute_UB(r1, r2)

1

In [11]:
# UB: 1.207702707 1.248454819 0.002095582696 
#     -1.485612421 0.9118074731 0.003241829804 
#     -0.0173752388 0.02282507942 1.651530555

# FIXME:  must get same numbers, probably in different rows

sixc.UB.get()

array([[-1.2466385 ,  1.00614404,  0.14993609],
       [ 0.06162735, -0.16202206,  1.59964532],
       [ 1.015386  ,  1.24512539,  0.08699568]])

In [12]:
print('calc.wavelength is', sixc.calc.wavelength)
print('sample is', sixc.calc.sample)
print('position is', sixc.position)

print('sample name is', sixc.sample_name.get())
print('u matrix is', sixc.U.get(), sixc.U.describe())
print('ub matrix is', sixc.UB.get(), sixc.UB.describe())
print('reflections:',
        sixc.reflections.get(),
        sixc.reflections.describe())
print('ux is', sixc.ux.get(), sixc.ux.describe())
print('uy is', sixc.uy.get(), sixc.uy.describe())
print('uz is', sixc.uz.get(), sixc.uz.describe())
print('lattice is', sixc.lattice.get(), sixc.lattice.describe())
print(sixc.read())

calc.wavelength is 0.8265616267
sample is HklSample(name='JL124_1', lattice=LatticeTuple(a=3.905, b=3.905, c=3.905, alpha=90.0, beta=90.0, gamma=90.0), ux=Parameter(name='None (internally: ux)', limits=(min=-180.0, max=180.0), value=-86.88707255123839, fit=True, inverted=False, units='Degree'), uy=Parameter(name='None (internally: uy)', limits=(min=-180.0, max=180.0), value=5.3468811222840165, fit=True, inverted=False, units='Degree'), uz=Parameter(name='None (internally: uz)', limits=(min=-180.0, max=180.0), value=-141.09349844908007, fit=True, inverted=False, units='Degree'), U=array([[-0.77478589,  0.62531858,  0.09318529],
       [ 0.0383014 , -0.10069672,  0.99417965],
       [ 0.63106246,  0.77384549,  0.05406782]]), UB=array([[-1.2466385 ,  1.00614404,  0.14993609],
       [ 0.06162735, -0.16202206,  1.59964532],
       [ 1.015386  ,  1.24512539,  0.08699568]]), reflections=[(h=0.0, k=0.0, l=2.0), (h=3.0, k=0.0, l=3.0)], reflection_measured_angles=array([[0.        , 1.23564414]